<a href="https://colab.research.google.com/github/Vedalaxman/Audio_to_Text/blob/main/Audio_To_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-generativeai

In [4]:
import os
import google.generativeai as genai

PROMPT_TEMPLATE = """
You are an expert medical scribe. Your task is to take a raw transcription of a doctor-patient consultation and structure it according to the Indian National EHR Standard format.

Analyze the following transcription and extract the information into the sections below.
- If information for a section is not present in the transcript, write "Not mentioned."
- Do not invent any information that is not explicitly stated in the transcription.

**[Patient Identification]**
- Name:
- Age:
- Gender:

**[Chief Complaints]**
-

**[History of Present Illness]**
-

**[Past Medical/Surgical History]**
-

**[Medication History]**
-

**[Allergies]**
-

**[Family History]**
-

**[Physical Examination Findings]**
-

**[Provisional Diagnosis / Assessment]**
-

**[Care Plan / Treatment]**
-

Here is the raw transcription to format:
---
{raw_transcription_text}
---
"""

def configure_api():
    api_key = "api_key"
    genai.configure(api_key=api_key)

def format_transcription(transcription_text: str) -> str:
    model = genai.GenerativeModel('gemini-2.0-flash')
    final_prompt = PROMPT_TEMPLATE.format(raw_transcription_text=transcription_text)
    response = model.generate_content(final_prompt)
    return response.text

if __name__ == "__main__":
    configure_api()

    audio_file_path = "/content/OSR_us_000_0010_8k.wav"

    print(f"Uploading file: {audio_file_path}")
    audio_file = genai.upload_file(path=audio_file_path)
    print(f"Completed upload: {audio_file.name}")

    model = genai.GenerativeModel(model_name="gemini-2.0-flash")
    prompt = "Transcribe this audio file and provide the full text."

    print("Generating content...")
    response = model.generate_content([prompt, audio_file])

    transcription_text = response.text

    print("\n--- Raw Transcription ---")
    print(transcription_text)

    print("\n--- Formatted EHR Output ---")
    formatted_output = format_transcription(transcription_text)
    print(formatted_output)


Uploading file: /content/OSR_us_000_0010_8k.wav
Completed upload: files/q0cdfgur8ie2
Generating content...

--- Raw Transcription ---
The birch canoe slid on the smooth planks. Glue the sheet to the dark blue background. It is easy to tell the depth of a well. These days, a chicken leg is a rare dish. Rice is often served in round bowls. The juice of lemons makes fine punch. The box was thrown beside the parked truck. The hogs were fed chopped corn and garbage. Four hours of steady work faced us. A large size in stockings is hard to sell.

--- Formatted EHR Output ---
This transcription appears to be a collection of random sentences and does not contain any medical information. Therefore, I cannot extract any relevant details to populate the fields of the Indian National EHR Standard format.

**[Patient Identification]**
- Name: Not mentioned.
- Age: Not mentioned.
- Gender: Not mentioned.

**[Chief Complaints]**
- Not mentioned.

**[History of Present Illness]**
- Not mentioned.

**[P

In [1]:
import os
import google.generativeai as genai

# --- 1. DEFINE YOUR PROMPT TEMPLATE ---
# This is the exact prompt you designed.
PROMPT_TEMPLATE = """
You are an expert medical scribe. Your task is to take a raw transcription of a doctor-patient consultation and structure it according to the Indian National EHR Standard format.

Analyze the following transcription and extract the information into the sections below.
- If information for a section is not present in the transcript, write "Not mentioned."
- Do not invent any information that is not explicitly stated in the transcription.

**[Patient Identification]**
- Name:
- Age:
- Gender:

**[Chief Complaints]**
- (List the primary reasons for the visit as stated by the patient)

**[History of Present Illness]**
- (Provide a detailed narrative of the current problem, including onset, duration, context, and symptoms)

**[Past Medical/Surgical History]**
- (List any past illnesses, surgeries, or chronic conditions)

**[Medication History]**
- (List current medications and dosages mentioned)

**[Allergies]**
- (List any mentioned allergies to drugs, food, etc.)

**[Family History]**
- (Mention any relevant family medical history if discussed)

**[Physical Examination Findings]**
- (Summarize any physical examination results or observations mentioned by the doctor)

**[Provisional Diagnosis / Assessment]**
- (List the diagnosis or assessment provided by the doctor)

**[Care Plan / Treatment]**
- (Detail the treatment plan, including prescriptions, tests ordered, advice, and follow-up instructions)


Here is the raw transcription to format:
---
{raw_transcription_text}
---
"""

def configure_api():
    """Configure the Generative AI API with the API key."""
    # Best practice: store your key in an environment variable
    api_key = os.getenv("")
    if not api_key:
        raise ValueError("API key not found. Please set the GOOGLE_API_KEY environment variable.")
    genai.configure(api_key=api_key)

def format_transcription(transcription_text: str) -> str:
    """
    Takes raw transcription text and uses an LLM to format it
    according to the predefined PROMPT_TEMPLATE.

    Args:
        transcription_text: A string containing the raw doctor-patient conversation.

    Returns:
        A formatted string with the extracted information.
    """
    model = genai.GenerativeModel('gemini-2.0-flash')
    final_prompt = PROMPT_TEMPLATE.format(raw_transcription_text=transcription_text)
    response = model.generate_content(final_prompt)
    return response.text
if __name__ == "__main__":

    # Configure the API key before making any calls
    try:
        configure_api()
    except ValueError as e:
        print(e)
        exit()
    raw_text = """
    Doctor: Good morning, please come in. What brings you here today?
    Patient: Good morning, Doctor. I've been having this really bad headache for the past three days. It's a throbbing pain, mostly on the right side of my head.
    Doctor: I see. Mr... Sharma, is it? Can you confirm your full name and age for me?
    Patient: Yes, it's Anil Sharma. I am 45 years old.
    Doctor: Okay, Mr. Sharma. Apart from the headache, any other symptoms? Nausea? Sensitivity to light?
    Patient: Yes, bright light makes it much worse. I've also felt a bit nauseous, but I haven't thrown up.
    Doctor: Any history of such headaches?
    Patient: Not like this. I get tension headaches sometimes, but this feels different. I had my appendix removed about 10 years ago, but otherwise, I'm healthy. I do take Metformin for my Type 2 Diabetes.
    Doctor: Understood. Any known allergies?
    Patient: None that I know of.
    Doctor: Okay. I'm going to check your blood pressure. *[pause]* It's a bit elevated at 140/90. Based on your symptoms - the one-sided pain, light sensitivity, and nausea - it's very likely a migraine. For now, I'm prescribing Tablet Sumatriptan 50mg, you can take one when a headache starts. I also want you to get a complete blood count test done, just to be sure. Please drink plenty of water and try to rest in a dark, quiet room. Follow up in one week.
    """

    print("--- 1. PROCESSING RAW TRANSCRIPTION ---")
    print(raw_text)
    print("\n" + "="*50 + "\n")

    # Call the function to get the formatted output
    formatted_output = format_transcription(raw_text)

    print("--- 2. GENERATED STRUCTURED OUTPUT ---")
    print(formatted_output)

API key not found. Please set the GOOGLE_API_KEY environment variable.
--- 1. PROCESSING RAW TRANSCRIPTION ---

    Doctor: Good morning, please come in. What brings you here today?
    Patient: Good morning, Doctor. I've been having this really bad headache for the past three days. It's a throbbing pain, mostly on the right side of my head.
    Doctor: I see. Mr... Sharma, is it? Can you confirm your full name and age for me?
    Patient: Yes, it's Anil Sharma. I am 45 years old.
    Doctor: Okay, Mr. Sharma. Apart from the headache, any other symptoms? Nausea? Sensitivity to light?
    Patient: Yes, bright light makes it much worse. I've also felt a bit nauseous, but I haven't thrown up.
    Doctor: Any history of such headaches?
    Patient: Not like this. I get tension headaches sometimes, but this feels different. I had my appendix removed about 10 years ago, but otherwise, I'm healthy. I do take Metformin for my Type 2 Diabetes.
    Doctor: Understood. Any known allergies?
    Pa

DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

In [ ]:

raw_text = """
    Doctor: Good morning, please come in. What brings you here today?
    Patient: Good morning, Doctor. I've been having this really bad headache for the past three days. It's a throbbing pain, mostly on the right side of my head.
    Doctor: I see. Mr... Sharma, is it? Can you confirm your full name and age for me?
    Patient: Yes, it's Anil Sharma. I am 45 years old.
    Doctor: Okay, Mr. Sharma. Apart from the headache, any other symptoms? Nausea? Sensitivity to light?
    Patient: Yes, bright light makes it much worse. I've also felt a bit nauseous, but I haven't thrown up.
    Doctor: Any history of such headaches?
    Patient: Not like this. I get tension headaches sometimes, but this feels different. I had my appendix removed about 10 years ago, but otherwise, I'm healthy. I do take Metformin for my Type 2 Diabetes.
    Doctor: Understood. Any known allergies?
    Patient: None that I know of.
    Doctor: Okay. I'm going to check your blood pressure. *[pause]* It's a bit elevated at 140/90. Based on your symptoms - the one-sided pain, light sensitivity, and nausea - it's very likely a migraine. For now, I'm prescribing Tablet Sumatriptan 50mg, you can take one when a headache starts. I also want you to get a complete blood count test done, just to be sure. Please drink plenty of water and try to rest in a dark, quiet room. Follow up in one week.
    """
print("--- 1. PROCESSING RAW TRANSCRIPTION ---")
print(raw_text)
print("\n" + "="*50 + "\n")

    # Call the function to get the formatted output
formatted_output = format_transcription(raw_text)

--- 1. PROCESSING RAW TRANSCRIPTION ---

    Doctor: Good morning, please come in. What brings you here today?
    Patient: Good morning, Doctor. I've been having this really bad headache for the past three days. It's a throbbing pain, mostly on the right side of my head.
    Doctor: I see. Mr... Sharma, is it? Can you confirm your full name and age for me?
    Patient: Yes, it's Anil Sharma. I am 45 years old.
    Doctor: Okay, Mr. Sharma. Apart from the headache, any other symptoms? Nausea? Sensitivity to light?
    Patient: Yes, bright light makes it much worse. I've also felt a bit nauseous, but I haven't thrown up.
    Doctor: Any history of such headaches?
    Patient: Not like this. I get tension headaches sometimes, but this feels different. I had my appendix removed about 10 years ago, but otherwise, I'm healthy. I do take Metformin for my Type 2 Diabetes.
    Doctor: Understood. Any known allergies?
    Patient: None that I know of.
    Doctor: Okay. I'm going to check your b

NameError: name 'format_transcription' is not defined